In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings

# import workalendar
# from workalendar.america import Brazil

warnings.filterwarnings('ignore')
np.random.seed(4590)


In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/historical_transactions.csv',parse_dates=['purchase_date'])
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv',parse_dates=['purchase_date'])

In [3]:
df_hist_trans['merchant_id'].fillna('-1',inplace=True)
df_new_merchant_trans['merchant_id'].fillna('-1',inplace=True)

In [4]:
all_ids = df_train['card_id'].tolist()+df_test['card_id'].tolist()+df_hist_trans['merchant_id'].tolist()+\
            df_new_merchant_trans['merchant_id'].tolist()
    
from sklearn.preprocessing import  LabelEncoder

lbe = LabelEncoder()
lbe.fit(all_ids)

df_train['card_id']=lbe.transform(df_train['card_id'])
df_test['card_id']=lbe.transform(df_test['card_id'])

df_hist_trans['card_id']= lbe.transform(df_hist_trans['card_id'])
df_new_merchant_trans['card_id']= lbe.transform(df_new_merchant_trans['card_id'])

df_hist_trans['merchant_id'] = lbe.transform(df_hist_trans['merchant_id'])
df_new_merchant_trans['merchant_id'] = lbe.transform(df_new_merchant_trans['merchant_id'])

In [5]:
card_id_2_merchant_dict = {}

In [6]:
for i in df_train['card_id'].values:
    card_id_2_merchant_dict[i]=set()
    
for i in df_test['card_id'].values:
    card_id_2_merchant_dict[i]=set()

In [7]:
for i in  range(len(df_hist_trans)):
    card_id = df_hist_trans['card_id'].values[i]
    merchant_id = df_hist_trans['merchant_id'].values[i]
    card_id_2_merchant_dict[card_id].add(merchant_id)
    
# for i in  range(len(df_new_merchant_trans)):
#     card_id = df_new_merchant_trans['card_id'].values[i]
#     merchant_id = df_new_merchant_trans['merchant_id'].values[i]
#     card_id_2_merchant_dict[card_id].add(merchant_id)

In [8]:
with open("carid_2_merchant_graph.list",'w') as f:
    for card_id in  card_id_2_merchant_dict.keys():
        for merchant_id in card_id_2_merchant_dict[card_id]:
            f.write(str(card_id)+" "+str(merchant_id)+"\n")